In [ ]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
import tensorflow_hub as hub
import keras
from keras.models import Model
#from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense,Input,BatchNormalization,Dropout
from keras.layers import LSTM
#from keras.optimizers import Adam,Adadelta
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from urllib.request import urlopen # the lib that handles the url stuff
import string
#import pickle
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
#from tensorflow.keras.callbacks import TensorBoard

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
corpus= 'https://dev.gutenberg.org/files/63677/63677-0.txt' # The Project Gutenberg EBook of The Recluse, by Mike Curry
corpus2= 'https://dev.gutenberg.org/files/63964/63964-0.txt' # TThe Project Gutenberg EBook of The Convict Ship, Volume 1 (of 3), by William Clark Russell

# Data Cleaning

data = urlopen(corpus,) # it's a file like object and works just like a file
clean_data=[x.decode('utf-8').strip() for x in data]

data2 = urlopen(corpus2,) # it's a file like object and works just like a file
clean_data2=[x.decode('utf-8').strip() for x in data2]
clean_data2=clean_data2[166:] #remove the early description in the text.
combined_data= clean_data+clean_data2

combined_data=list(filter(None, combined_data))
cleaned=' '.join(combined_data)
cleaned=cleaned.translate(str.maketrans('', '', string.punctuation)) #remve punctuation

In [ ]:
tokens = word_tokenize(cleaned)
train_len = 4
text_sequences = []
for i in range(train_len,len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

sequences = tokenizer.texts_to_sequences(text_sequences)
#vocabulary size increased by 1 for the cause of padding
vocabulary_size = len(tokenizer.word_counts)+1
n_sequences = np.empty([len(sequences),train_len], dtype='float64')
for i in range(len(sequences)):
  n_sequences[i] = sequences[i]
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
#train_targets = to_categorical(train_targets, num_classes=vocabulary_size

In [ ]:
#text_sequences
train_texts=[]
target_texts=[]
for i in text_sequences:
  train_texts.append(i[:-1]) 
  target_texts.append(i[-1])

train_txt=[]
for text in train_texts:  
  my_lst_str = [' '.join(map(str,text))]
  train_txt.append(my_lst_str)


In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.load(module_url)

In [ ]:
train_text=[sentence[0] for sentence in train_txt]


In [ ]:
len(train_txt)

67724

In [ ]:
embeddings_train = embed(train_text)


In [ ]:
y_train=pd.get_dummies(train_targets).values

In [ ]:
model=Sequential()
model.add(Dense(256, activation='relu',input_shape=(512,)))
model.add(tf.keras.layers.Reshape((128, 2)))
model.add( Dropout(0.4))
model.add(BatchNormalization())
model.add(LSTM(128))
model.add(Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add( Dense(y_train.shape[1],activation='softmax',name='output'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
reshape (Reshape)            (None, 128, 2)            0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 2)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 2)            8         
_________________________________________________________________
lstm (LSTM)                  (None, 128)               67072     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [ ]:
# clean up as much as possible
import gc
print(gc.collect())

4431


In [ ]:
model.compile(optimizer='Adam', loss='CategoricalCrossentropy')

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(embeddings_train ,y_train,validation_split=0.2, epochs=20,batch_size=64,callbacks=[callback])

model.save('pred_model4.h5')

Epoch 1/20
847/847 [==============================] - 184s 215ms/step - loss: 7.5307 - val_loss: 6.5248
Epoch 2/20
847/847 [==============================] - 176s 208ms/step - loss: 6.1922 - val_loss: 6.5475
Epoch 3/20
847/847 [==============================] - 176s 207ms/step - loss: 6.0902 - val_loss: 6.4597
Epoch 4/20
847/847 [==============================] - 178s 210ms/step - loss: 6.0261 - val_loss: 6.4100
Epoch 5/20
847/847 [==============================] - 180s 213ms/step - loss: 5.9757 - val_loss: 6.3709
Epoch 6/20
847/847 [==============================] - 172s 203ms/step - loss: 5.9000 - val_loss: 6.3770
Epoch 7/20
847/847 [==============================] - 177s 208ms/step - loss: 5.8751 - val_loss: 6.3507
Epoch 8/20
847/847 [==============================] - 176s 208ms/step - loss: 5.8048 - val_loss: 6.4092
Epoch 9/20
847/847 [==============================] - 176s 207ms/step - loss: 5.7754 - val_loss: 6.3968
Epoch 10/20
847/847 [==============================] - 174s 206m